In [86]:
import pandas as pd
import yaml
import os
import plotly.express as px
import plotly
from plotly import graph_objects as go

In [87]:
max_dict = {
    'ar': 6074,
    'pl': 17721,
    'bg': 8906,
    'be': 22852,
    'fr': 14448,
    'en': 12542,
    'lv': 12520,
    'fa': 4797,
    'lt': 2340,
    'cs': 68494,
    'es': 14286, 
    'zh': 3996,
    'slk': 8482,
    'slv': 10902,
    'uk': 5495,
    'ru': 69629,
    'de': 13813,
    'fi': 12216,
    'id': 4481,
    'hr': 6913,
}

In [88]:
slavic_langs = ("pl", "be", "cs", "slk", "slv", "uk", "ru")

In [89]:
RESULTS_CSV = '/home/amysiak/thesis/multilingual-probing-visualization/probingOutputs/all_slavic/all_results.csv'

In [90]:
df = pd.read_csv(RESULTS_CSV)
print(df.columns)
print(df.dtypes)

Index(['train', 'test', 'model', 'layer', 'rank', 'limit', 'seed', 'directory',
       'dspear', 'uuas'],
      dtype='object')
train         object
test          object
model         object
layer          int64
rank           int64
limit        float64
seed           int64
directory     object
dspear       float64
uuas         float64
dtype: object


In [91]:
df['train'] = df.train.apply(lambda x: x[1:-1].replace("'", "").split(',')[0])
df['test'] = df.test.apply(lambda x: x[1:-1].replace("'", "").split(',')[0])
df["slavic"] = df.train.isin(slavic_langs)
df['limit'] = df.apply(lambda x: x.limit if not pd.isna(x.limit) else max_dict[x.train], axis=1)
df.head()

,train,test,model,layer,rank,limit,seed,directory,dspear,uuas,slavic
0,be,be,mBERT,6,128,100.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.416957,0.397976,True
1,be,be,mBERT,6,128,1000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.711804,0.634634,True
2,be,be,mBERT,6,128,10000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.779117,0.787562,True
3,be,be,mBERT,6,128,2500.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.756653,0.720526,True
4,be,be,mBERT,6,128,5000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.770321,0.756249,True


In [92]:
avg_df = df.groupby(['train', 'test', 'model', 'layer', 'rank', 'limit']).mean().reset_index().drop(columns=["seed"])
avg_df["slavic"] = avg_df.slavic.astype(bool)

/tmp/ipykernel_1096820/2098689671.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [93]:
avg_df.head()

,train,test,model,layer,rank,limit,dspear,uuas,slavic
0,be,be,mBERT,6,128,100.0,0.424506,0.400528,True
1,be,be,mBERT,6,128,1000.0,0.714579,0.638396,True
2,be,be,mBERT,6,128,2500.0,0.752713,0.709843,True
3,be,be,mBERT,6,128,5000.0,0.770202,0.757244,True
4,be,be,mBERT,6,128,7500.0,0.775498,0.779258,True


In [94]:
for test in avg_df.test.unique():
    df_loc = avg_df[(avg_df.test == test) & (avg_df.limit <= 10000)]
    fig = px.scatter(data_frame=df_loc, x="limit", y="uuas", color="slavic", hover_data=["train"], labels={"limit": "# train sentences", "uuas": "UUAS", "slavic": "Is Slavic"}, title=f"{test}")
    fig.update_layout(
        autosize=False,
        width=800,
        height=600,
        yaxis_range=[0, 1],
        font={"size":20}
    )
    fig.show()

In [ ]:
for test in avg_df.test.unique():
    df_loc = avg_df[(avg_df.test == test)]
    fig = px.line(data_frame=df_loc, x="limit", y="uuas", color="train", markers=True)
    fig.show()